<font size = "6"> Fordham Sports Analytics Society Big Data Bowl 2023 - Feature Creation </font>

<font size = "4"> Convert data into one cleaned data frame that can be used in model creation. </font>

- Authors:  Peter Majors, Chris Orlando, Jack Townsend, and Etienne Busnel
- Kaggle:  https://www.kaggle.com/competitions/nfl-big-data-bowl-2023/overview (Resources)
- Our Github:  https://github.com/peterlmajors/FSAS_BigDataBowl_2023 (Up-To-Date Code)

In [36]:
#Import Required Packages

#Data Manipulation
import pandas as pd
import numpy as np
import math

#Data Vizualization
import seaborn as sns 
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from haversine import haversine

#Notebook Settings
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [37]:
#Import Our Merged DataFrames
#ptrack = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/ptrack.csv") #Player Tracking
ptrack_qb_poss = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/ptrack_qb_poss.csv") #Player Tracking Only On Frames Where QB The Target For Pass Rushers
pbp = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/pbp.csv") #Play-By-Play Data
pff_qb_pressure = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/pff_qb_pressure.csv") #Import PFF Data On QB Pressures

<font size="5"> Feature 1: Speed and Acceleration of A Pass Rusher Coming Into A Block </font>

 - With an Immediate Zone 1.5 Yards In Depth and 1.5 Yards Across
 
 - Speed More Likely To Be Used For Tackles, While Guards And Centers Will Almost Definitely Used Acceleration

    -   Tackles =    Avg Yards From Rusher At Ball Snap

    -   Guards & Centers =    Avg Yards From Rusher At Ball Snap

- Speed and Acceleration On First Frame In The Immediate Zone

- How Explosive Was The Pass Rusher On The Selected Frame?

In [38]:
#Create Data Frame With Only Pass Blocking Plays and Where The Player Rusher Is Known And Is A Pass Rusher 
#These Are All The Columns For Which The Immediate Zone Was Calculated And Where The QB Has Possession of The Football
ptrack_qb_poss_block = ptrack_qb_poss.loc[(ptrack_qb_poss['pff_blockType'] != '0') & (ptrack_qb_poss['pff_nflIdBlockedPlayer'] == ptrack_qb_poss['nflId_rusher'])]

#Create Composite PK of gameId, playId, and nflId 
ptrack_qb_poss_block[['gameId', 'playId', 'nflId']] = ptrack_qb_poss_block[['gameId', 'playId', 'nflId']].astype(str)
ptrack_qb_poss_block['game_play_nfl_Id'] = ptrack_qb_poss_block[["gameId", "playId", 'nflId']].apply("-".join, axis=1)

C:\Users\Peter\AppData\Local\Temp\ipykernel_11692\2514964046.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptrack_qb_poss_block[['gameId', 'playId', 'nflId']] = ptrack_qb_poss_block[['gameId', 'playId', 'nflId']].astype(str)
C:\Users\Peter\AppData\Local\Temp\ipykernel_11692\2514964046.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptrack_qb_poss_block['game_play_nfl_Id'] = ptrack_qb_poss_block[["gameId", "playId", 'nflId']].apply("-".join, axis=1)


In [121]:
#For Each Play For A Blocker, Find The Speed & Of The Rusher At The First Frame In The Immediate Zone (Very Long Run Time)

#Find All Play PKs
play_blocker_pk = ptrack_qb_poss_block.game_play_nfl_Id.unique()
first_frame_imm_zone = []

#Loop Through All 
for i in range(len(play_blocker_pk)):
 
    #Initialize Variables For Each Iteration
    max_frames = ptrack_qb_poss_block[ptrack_qb_poss_block['game_play_nfl_Id'] == play_blocker_pk[i]].shape[0] - 1    
    frame = 0

    #Add Rows Where The First Frame In The Immediate Zone Happens
    while (frame <= max_frames):
        frame = 
        if ptrack_qb_poss_block[ptrack_qb_poss_block['game_play_nfl_Id'] == play_blocker_pk[i]]['rusher_in_imm_box'].reset_index().at[frame,'rusher_in_imm_box'] == 1:
            #Append Play PK To A List
            first_frame_imm_pk = ptrack_qb_poss_block.loc[(ptrack_qb_poss_block['game_play_nfl_Id'] == play_blocker_pk[i]) & (ptrack_qb_poss_block['frameId'] == frame)]['game_play_nfl_Id'].iloc[0]
            first_frame_imm_zone.append(first_frame_imm_pk)
            break
        else:
            frame += 1  

IndexError: single positional indexer is out-of-bounds

In [131]:
ptrack_qb_poss_block[ptrack_qb_poss_block['game_play_nfl_Id'] == play_blocker_pk[0]]['rusher_in_imm_box'].reset_index()

# .at[4,'rusher_in_imm_box']

,index,rusher_in_imm_box
0,4,0.0
1,26,0.0
2,48,0.0
3,70,0.0
4,92,1.0
5,114,1.0
6,136,1.0
7,158,1.0
8,180,1.0
9,202,0.0


In [132]:
ptrack_qb_poss_block.loc[(ptrack_qb_poss_block['game_play_nfl_Id'] == play_blocker_pk[0])] 

# & (ptrack_qb_poss_block['frameId'] == 4)]

# ['game_play_nfl_Id']



,Unnamed: 0,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,height,weight,birthDate,collegeName,officialPosition,displayName,x_qb,y_qb,s_qb,a_qb,dis_qb,o_qb,dir_qb,dist_from_qb,angle_to_qb,angle_to_qb_diff_o,nflId_blocker,nflId_rusher,displayName_rusher,pff_positionLinedUp_rusher,x_rusher,y_rusher,s_rusher,a_rusher,dis_rusher,o_rusher,dir_rusher,rusher_in_imm_box,blocker_rusher_distance,diff_btw_rusher_dir_blocker_o,frame_first,frame_last,game_play_nfl_Id
4,4,2021090900,97,40151,Pass Block,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44955.0,SW,0.0,6,2021-09-10T00:26:31.600,66.0,TB,right,42.10,24.02,0.51,1.98,0.06,48.28,308.34,ball_snap,6-4,319,1991-05-27,Colorado State-Pueblo,C,Ryan Jensen,37.64,24.26,0.35,0.53,0.05,144.42,282.72,4.466453,89.83,-41.55,40151.0,44955.0,Carlos Watkins,DRT,43.35,25.18,0.56,2.42,0.06,243.27,288.42,0.0,1.71,60.14,6,40,2021090900-97-40151
26,26,2021090900,97,40151,Pass Block,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44955.0,SW,0.0,7,2021-09-10T00:26:31.700,66.0,TB,right,42.03,24.06,0.79,2.13,0.07,53.21,305.04,None,6-4,319,1991-05-27,Colorado State-Pueblo,C,Ryan Jensen,37.56,24.26,0.54,1.05,0.08,137.49,272.95,4.474472,89.89,-36.68,40151.0,44955.0,Carlos Watkins,DRT,43.28,25.20,0.88,2.45,0.08,233.66,288.73,0.0,1.69,55.52,6,40,2021090900-97-40151
48,48,2021090900,97,40151,Pass Block,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44955.0,SW,0.0,8,2021-09-10T00:26:31.800,66.0,TB,right,41.95,24.10,1.00,1.84,0.09,57.07,300.02,None,6-4,319,1991-05-27,Colorado State-Pueblo,C,Ryan Jensen,37.47,24.25,0.80,1.85,0.09,131.95,267.49,4.482510,89.94,-32.87,40151.0,44955.0,Carlos Watkins,DRT,43.13,25.25,1.39,2.70,0.15,231.10,287.62,0.0,1.65,50.55,6,40,2021090900-97-40151
70,70,2021090900,97,40151,Pass Block,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44955.0,SW,0.0,9,2021-09-10T00:26:31.900,66.0,TB,right,41.86,24.16,1.17,1.35,0.11,59.92,301.28,None,6-4,319,1991-05-27,Colorado State-Pueblo,C,Ryan Jensen,37.38,24.24,0.99,2.03,0.09,129.85,263.48,4.480714,89.98,-30.06,40151.0,44955.0,Carlos Watkins,DRT,42.96,25.28,1.74,2.24,0.18,228.58,284.85,0.0,1.57,44.93,6,40,2021090900-97-40151
92,92,2021090900,97,40151,Pass Block,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44955.0,SW,0.0,10,2021-09-10T00:26:32.000,66.0,TB,right,41.74,24.24,1.39,1.09,0.14,62.96,301.82,None,6-4,319,1991-05-27,Colorado State-Pueblo,C,Ryan Jensen,37.27,24.23,1.19,1.82,0.11,123.79,263.77,4.470011,90.00,-27.04,40151.0,44955.0,Carlos Watkins,DRT,42.76,25.31,1.99,1.41,0.20,215.87,282.18,1.0,1.48,39.22,6,40,2021090900-97-40151
114,114,2021090900,97,40151,Pass Block,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44955.0,SW,0.0,11,2021-09-10T00:26:32.100,66.0,TB,right,41.58,24.31,1.62,0.98,0.18,71.03,297.56,None,6-4,319,1991-05-27,Colorado State-Pueblo,C,Ryan Jensen,37.14,24.22,1.38,1.61,0.13,119.88,261.95,4.440912,89.98,-18.95,40151.0,44955.0,Carlos Watkins,DRT,42.55,25.37,2.13,0.79,0.21,220.11,283.67,1.0,1.44,32.64,6,40,2021090900-97-40151
136,136,2021090900,97,40151,Pass Block,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44955.0,SW,0.0,12,2021-09-10T00:26:32.200,66.0,TB,right,41.42,24.37,1.72,0.67,0.17,77.20,293.88,None,6-4,319,1991-05-27,Colorado State-Pueblo,C,Ryan Jensen,36.99,24.19,1.57,1.43,0.15,113.65,260.27,4.433655,89.91,-12.71,40151.0,44955.0,Carlos Watkins,DRT,42.35,25.40,2.14,0.20,0.21,225.22,281.67,1.0,1.39,24.47,6,40,2021090900-97-40151
158,158,2021090900,97,40151,Pass Block,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44955.0,SW,0.0,13,2021-09-10T00:26:32.300,66.0,TB,right,41.23,24.43,1.87,0.63,0.20,79.51,289.71,None,6-4,319,1991-05-27,Colorado State-Pueblo,C,Ryan Jensen,36.80,24.16,1.82,1.37,0.19,107.99,261.01,4.438220,89.79,-10.28,40151.0,44955.0,Carlos Watkins,DRT,42.15,25.44,2.10,0.52,0.21,234.60,280.90,1.0,1.37,21.39,6,40,2021090900-97-40151
180,180,2021090900,97,40151,Pass Block,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44955.0,SW,0.0,14,2021-09-10

In [120]:
ptrack_qb_poss_block.loc[(ptrack_qb_poss_block['game_play_nfl_Id'] == play_blocker_pk[0]) & (ptrack_qb_poss_block['frameId'] == frame)]

,Unnamed: 0,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,height,weight,birthDate,collegeName,officialPosition,displayName,x_qb,y_qb,s_qb,a_qb,dis_qb,o_qb,dir_qb,dist_from_qb,angle_to_qb,angle_to_qb_diff_o,nflId_blocker,nflId_rusher,displayName_rusher,pff_positionLinedUp_rusher,x_rusher,y_rusher,s_rusher,a_rusher,dis_rusher,o_rusher,dir_rusher,rusher_in_imm_box,blocker_rusher_distance,diff_btw_rusher_dir_blocker_o,frame_first,frame_last,game_play_nfl_Id


In [116]:
first_frame_imm_zone = []
first_frame_imm_zone.append(test)
first_frame_imm_zone

['2021090900-97-52421']

In [ ]:

#Filter The ptrack_qb_poss_block Down To Rows That Include The Row PKs Appended Above
ptrack_qb_poss_block['first_frame_imm_zone'] = np.where(ptrack_qb_poss_block.game_play_nfl_Id.isin(first_frame_imm_zone) == True, 1, 0)

In [66]:
ptrack_qb_poss_block.loc[(ptrack_qb_poss_block['game_play_nfl_Id'] == play_blocker_pk[7]) & (ptrack_qb_poss_block['frameId'] == 6)]

,Unnamed: 0,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,height,weight,birthDate,collegeName,officialPosition,displayName,x_qb,y_qb,s_qb,a_qb,dis_qb,o_qb,dir_qb,dist_from_qb,angle_to_qb,angle_to_qb_diff_o,nflId_blocker,nflId_rusher,displayName_rusher,pff_positionLinedUp_rusher,x_rusher,y_rusher,s_rusher,a_rusher,dis_rusher,o_rusher,dir_rusher,rusher_in_imm_box,blocker_rusher_distance,diff_btw_rusher_dir_blocker_o,frame_first,frame_last,game_play_nfl_Id
778,778,2021090900,410,42404,Pass Block,LG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42403.0,PP,0.0,6,2021-09-10T00:38:23.300,74.0,TB,right,19.45,28.42,0.0,0.0,0.0,84.72,298.03,ball_snap,6-4,307,1993-04-17,Hobart,G,Ali Marpet,15.88,26.97,0.0,0.0,0.01,180.77,63.08,3.853232,80.63,4.09,42404.0,42403.0,Randy Gregory,RE,21.32,29.79,0.03,0.27,0.01,286.36,234.75,0.0,2.32,29.97,6,22,2021090900-410-42404


In [ ]:
#Take Take The New Data And Merge Onto The Main
first_frame_imm_zone = [['game_play_nfl_Id', 's_rusher', 'a_rusher', 'diff_btw_rusher_dir_blocker_o', ]]
first_frame_imm_zone = first_frame_imm_zone.rename(column = {'diff_btw_rusher_dir_blocker_o': 'diff_btw_rusher_dir_blocker_o_first_imm_zone'})
ptrack_qb_poss_block = ptrack_qb_poss_block.merge(first_frame_imm_zone, on = 'game_play_nfl_Id', how = 'left')

#Fill The Null Values With The Average For The Position
#First Find The Average For Each Speed and Acceleration For Each Position
speed_avg_pos_first_frame_imm_zone = []
acc_avg_pos_first_frame_imm_zone = []
for i in range(len(ptrack_qb_poss_block.pff_positionLinedUp.unique())):
    ptrack_qb_poss_block.loc['']

ptrack_qb_poss_block.diff_btw_rusher_dir_blocker_o

In [ ]:
#Show All fff
ptrack_qb_poss_block.loc[(ptrack_qb_poss_block['gameId'] == 2021090900) & (ptrack_qb_poss_block['playId'] == 97)]['nflId'].value_counts()

In [ ]:
first_frame_imm_zone.loc[(first_frame_imm_zone['gameId'] == 2021090900) & (first_frame_imm_zone['playId'] == 97) & (first_frame_imm_zone['nflId'] == 42377)][['gameId','playId','nflId', 'frameId', 'rusher_in_imm_box', 'blocker_rusher_distance', 's']]

In [ ]:
ptrack_qb_poss_block.loc[(ptrack_qb_poss_block['gameId'] == 2021090900) & (ptrack_qb_poss_block['playId'] == 97) & (ptrack_qb_poss_block['nflId'] == 42377)][['gameId','playId','nflId', 'frameId', 'rusher_in_imm_box', 'blocker_rusher_distance', 's']]	

<font size="5"> Feature 2: Distance Traveled From Start To End Of Immediate Zone To QB </font>

- Attempts To Answer The Question of How Much A Pass Blocker Was "Pushed" During Their Handling Of A Rusher

- What Is The Raw Stopping Power of Our Offensive Linemen?

<font size="5"> Feature 3: Difference In Orientation Of Pass Blocker Between First And Last Frame Of Rusher Being In Immediate Zone </font>

- Attempts to Answer How Much An Offensive Lineman Was "Spun" In By The Pass Rusher

- The Difference Is Measured In Absolute Value, So The Change In Orientation Is Agnostic To Direction "Spun"

- How Well Can A Pass Blocker Hold Shoulder Angle When Engaged?

<font size="5"> Feature 4: Time The Pass Rusher Was Initially Kept In The Immediate Zone </font>

- For Frames When The Rusher Enters The Immediate Zone Multiple Times In A Play, The Number of Frames They Were In The Immediate Zone The First Time

- If The Rusher Only Enters The Immediate Zone Once In A Play, The Total Number of Seconds

- How Long Can A Pass Rusher Initially Maintain A Pass Rusher In Their Immediate Zone?

<font size="5"> Feature 5: Time After Initially Leaving The Immediate Zone That A Rusher Is In The Immediate Zone Again   </font>

- For Frames When The Rusher Enters The Immediate Zone Multiple Times In A Play, The Total Number of Frames They Were In The Immediate Zone For The Remainder Of The Play

- If The Rusher Only Enters The Immediate Zone Once In A Play, This Feature Is Left Null

- Acts As Insurance In Case A Rusher Comes Out Of The Immediate Zone For A Frame Or Two But Is Still Engaged With The Blocker

- How Well Can Our Pass Blocker Recover On Their Target If They Originally "Failed" In Keeping Them In Front of Them?

<font size="5"> Feature 6: Block Type Employed By The Pass Blocker </font>

- Aside From The Context Of Field Position, Block Types Can Have A Noticable Impact On The Distance A Rusher Gets To A Quarterback

- Allows Us To Control For Pass Blockers Who Are Routinely Given Easier or More Difficult Blocking Assignments As It Pertains To Rusher Distance From Quarterback

<font size="5"> Feature 7: Pressure To Sack Rate - <i> Courtesy of PFF Premium </i> </font>

- 

- Allows Us To Control For Pass Blockers Who Are Routinely Given Easier or More Difficult Blocking Assignments As It Pertains To Rusher Distance From Quarterback